# 코드 업데이트 이력
## 2022-10-26
  * Shot별 대푯값에 대한 통계 데이터 저장 테이블 변경 사항 반영 : tbShotResult → tbShotStatistics
  
## 2022-10-31
  * 전체 shot에 대한 유효 shot 도출 & 대푯값 추출 처리 개선
    - 문제 현상 : DB에서 전체 shot no list를 가져온 후 machine id가 일치하는 shot을 검색하여 별도의 리스트에 담을 때 이유를 알 수 없지만 중간에 멈추는 현상이 발생하여, shot 정보를 추출할 수 없는 상황이 반복됨
    - 처리 내용 : DB에서 전체 shot no list를 가져온 후 loop를 돌면서 machine id가 일치하는 shot에 대해 바로 유효 shot 도출 & 대푯값 추출 처리를 수행하도록 처리
  * 데이터 추출 시 tbShotStatistics에 누적된 데이터가 있는 경우, 이후 shot에 대한 유효 shot 도출 & 대푯값 추출 처리 개선
    - 문제 현상 : 마지막 shot 이후로 추출할 대상이 많은 경우(예를 들어 2개월분 데이터) "SELECT DISTIINCT shot_no where~" 절 사용 시 응답없음 수준으로 시간이 많이 소요됨
    - 처리 내용 : DB에서 전체 shot no list를 가져온 후 loop를 돌면서 machine id가 일치하는 shot 중 마지막으로 대푯값을 추출한 shot 이후의 shot에 대해 유효 shot 도출 & 대푯값 추출 처리

# 준비

In [376]:
import pandas as pd
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import random
import gc #garbage collector
import joblib
from sqlalchemy import create_engine
warnings.filterwarnings('ignore')

In [377]:
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_row', 200)

# 함수 정의

## 일반

### 컬럼 데이터 형변환 (64bit → 32bit)
 * 데이터 용량 줄이기 위한 용도

In [378]:
#주어진 dataframe에서 데이터 타입이 64bit인 경우 32bit로 변경하여 데이터 용량 축소
def change_data_type_64bit_to_32bit(data):
    data_type_names = list(data.columns.values)
    
    for col in data_type_names:
        data_type = str(data[col].dtype)
        
        if data_type == 'int64':
            data[col] = data[col].fillna(0).astype('int32')
        elif data_type == 'float64':
            data[col] = data[col].fillna(0).astype('float32')
        #LJY_20220929 : 컬럼의 data type이 'object'인데 실제 값은 numeric인 경우 int32로 변환
        # 추후 데이터 타입 판별로 변경 필요 (numeric인지 체크하는 api에서 datetime을 걸러내지 못해서 임시로 column name으로 체크하도록 처리)
        else:
            if col != "datetime" and col != "date":
                data[col] = data[col].replace('\\0', 0)
                data[col] = data[col].fillna(0).astype('int32')
#                data[col] = (data[col].fillna(0).astype('int32').astype(object).where(data[col].notnull()))
#                data[col] = data[col].notnull().astype('int32')
    
    data_type_names.clear()
    del data_type_names

### tool 정보 추출
 * 'tool_number' 필드에서 공구 정보를 추출하여 별도의 컬럼으로 구성

In [379]:
# tool 번호 및 상태 생성
def extract_tool_info_from_data(data):
    if 'tool_number' not in data.columns:
        return
    
    if data['tool_number'] is np.NaN:
        return
    
    data['tool_state'] = data['tool_number']%100
    data['tool'] = (data['tool_number'] - data['tool_state'])/100
    data['tool'] = data['tool'].astype(np.int32)

### 유효한 모델 파일 저장 경로 생성
  * 해당하는 폴더 또는 파일이 존재하지 않는 경우 ""를 반환

In [380]:
def make_valid_model_file_path(engine, model_name="Isolation Forest", machine_id=22, tool_no=5,
                               data_type_name='current_spindle', stat_type_name='data_mean'):
    #설비 이름을 DB에서 조회
    machine_name = get_machine_name_from_db(engine, machine_id)
    
    #설비 이름을 가져오지 못한 경우 아무 것도 하지 않고 return
    if len(machine_name) < 1:
        return ""
    
    #-----------------------------------------------------------------
    # 모델 파일을 불러올 경로 생성 & 유효성 체크
    path = ".\model"
    if os.path.isdir(path) == False:
        return ""
    
    path = path + "\\" + model_name
    if os.path.isdir(path) == False:
        return ""

    path = path + "\\" + machine_name
    if os.path.isdir(path) == False:
        return ""

    path = path + "\\" + "Tool{0}".format(tool_no)
    if os.path.isdir(path) == False:
        return ""
    
    stat_name = stat_type_name
    stat_name = stat_name.replace('data_', '')
    
    path = path + "\\" + "model_for_{0}_{1}.pkl".format(data_type_name, stat_name)
    print(path)
    if os.path.isfile(path) == False:
        return ""
    #-----------------------------------------------------------------
    
    return path

## 전처리 : 유효한 shot 정보 도출
 * 'shot_no' 필드 값을 기준으로 shot 구간 찾기
 * shot 구간 내부의 공구 사용 순서 및 각 공구 별 데이터 index 구간 도출
 * 공구 사용 순서에서 유효한 공구 사용 패턴 구간 찾기
 * 유효한 공구 사용 패턴 구간에 대해 numbering하여 별도의 'real_shot' 필드에 값 설정 (디폴트 : -1) 

### dataframe에 사용된 공구 순서 도출

In [381]:
#dataframe에 사용된 공구를 순서대로 추출해서 list형태로 반환
def extract_tool_list(data, report_result=False):
    import itertools

    tool_list = []
    #print([(k, (g)) for k, g in itertools.groupby(df_shot_specific['tool'])])
    for k, g in itertools.groupby(data['tool']):
        tool_list.append(k)

    if report_result is True:
        print(tool_list)
    
    return tool_list

In [382]:
#dataframe에 사용된 공구를 순서대로 추출해서 list형태로 반환
def extract_tool_list_with_range(data, report_result=False):
    import itertools

    irow = 0
    tool_list = []
    range_list_of_tool = []
    
    #print([(k, (g)) for k, g in itertools.groupby(df_shot_specific['tool'])])
    for k, g in itertools.groupby(data['tool']):
        tool_list.append(k)
        nrow = len(list(g))
        range_list_of_tool.append((irow,irow+nrow-1))
        irow += nrow

    if report_result is True:
        print(tool_list)
        print(range_list_of_tool)
    
    return tool_list, range_list_of_tool

In [383]:
#dataframe에서 (1) 주어진 shot_no에 해당하는 데이터를 추출한 후, (2) 해당 데이터에 사용된 공구를 순서대로 가져와서 반환
def extract_tool_list_of_specific_shot(data, shot_no, report_result=False):
    df_shot = data[data['shot_no'] == shot_no]
    
    if report_result is True:
        print("===== shot{0} ============================\n\n(1) dataframe -----------------\n".format(shot_no))
        print(df_shot)
        print("(2) used tool -----------------\n")
        print(df_shot['tool'].unique())
        print("(3) extract order using tools -----------------\n")
    
    tool_list = extract_tool_list(df_shot, report_result)
    
    del df_shot
    
    return tool_list

In [384]:
#dataframe에서 전체 shot에 대해 아래의 과정 수행
#   - 각 shot 별로 사용된 공구 순서를 추출
#   - shot & 공구 사용 순서정보를 column으로 하는 dataframe을 구성한 후 반환
def extract_tool_list_of_all_shots(data, report_result=False):
    data_type_names = ['shot_no', 'order_using_tool']
    df_by_shot = pd.DataFrame(columns=data_type_names)
    row = 0

    for shot in df['shot_no'].unique():
        tool_list = extract_tool_list_of_specific_shot(df, shot, report_result)
        str_tool_list = ' '.join(map(str, tool_list))
        df_by_shot.loc[row] = [shot, str_tool_list]
        
        del tool_list
        
        row += 1

    print(df_by_shot)
    
    return df_by_shot

### 유효 shot 구간 도출
 * 정상 공구 사용 패턴을 가지는 구간을 하나의 유효 shot으로 판별하고 별도의 column('real_shot')에 해당 numbering

In [385]:
def find_sub_list(sl,l):
    results=[]
    sll=len(sl)
    for ind in (i for i,e in enumerate(l) if e==sl[0]):
        if l[ind:ind+sll]==sl:
            results.append((ind,ind+sll-1))

    return results

In [386]:
#dataframe에서 (1) 주어진 shot_no에 해당하는 데이터를 추출한 후
#(2) 해당 데이터에 사용된 공구 순서 및 각 공구 별 row index 범위를 순서대로 가져옴
#(3) 정상 패턴 가공이 이루어지는 구간을 도출 (유효한 shot으로 판단할 수 있는 구간)
#(4) (3)에서 찾은 구간 데이터의 'real_shot' 컬럼에 새로 shot numbering
def find_and_mark_valid_shot_of_specific_shot(data, shot_no, real_shot_no, report_result=False):
    df_shot = data[data['shot_no'] == shot_no]
    
    if report_result is True:
        print("===== shot{0} ============================\n\n(1) dataframe -----------------\n".format(shot_no))
        print(df_shot)
        print("(2) used tool -----------------\n")
        print(df_shot['tool'].unique())
        print("(3) extract order using tools -----------------\n")
    
    tool_list, range_list_of_tool = extract_tool_list_with_range(df_shot, report_result=False)

    tool_pattern = [1, 5, 9, 11, 7, 3, 11, 7]
    sub_list = find_sub_list(tool_pattern, tool_list)
    
    if len(sub_list) < 1:
        if report_result is True:
            print("shot_no {0} of raw data : invalid_shot".format(shot_no))

        return 0
    
    if report_result is True:
        print(sub_list)
    
    start_index = df_shot.index[0]
    count = 0
    
    if report_result is True: 
        if sub_list[0][0] > 0:
            print("------ invalid data\n")
            print(data.iloc[start_index: start_index+range_list_of_tool[sub_list[0][0]][0]])
        
    for sub in sub_list:
        real_shot_no += 1
        sub_start = start_index + range_list_of_tool[sub[0]][0]
        sub_end   = start_index + range_list_of_tool[sub[1]][1]
        
        if report_result is True:
            print("sub({0}:{1}) - real_shot_no {2}".format(sub_start, sub_end, real_shot_no))
        
        #========================================================================
        #LJY_20220926 : iloc 함수를 사용하여 구간 access 후 값 설정 시 pandas ver.1.3.5 이후로(1.4.0부터) 동작하지 않던 오류 수정
        #------------------------------------------------------------
        #오류 : 기존에 작성된 코드
        #data.iloc[sub_start:sub_end]['real_shot'] = real_shot_no
        
        #------------------------------------------------------------
        #방법1 : column을 특정한 후 row index range 지정하여 값 설정
        #data.real_shot.iloc[sub_start:sub_end] = real_shot_no
        
        #------------------------------------------------------------
        #방법2 : row index range와 column index를 지정하여 값 설정
        data.iloc[sub_start:sub_end+1, data.columns.get_loc('real_shot')] = real_shot_no
        #========================================================================
        
        if report_result is True:
            print("------ valid shot[{0}]\n".format(count))
            print(data.iloc[sub_start:sub_end+1])
            
        count += 1
    
    tool_list.clear()
    range_list_of_tool.clear()
    sub_list.clear()
    
    del df_shot
    
    return count

In [387]:
#dataframe에서 유효한 공구 사용 패턴을 가지는 구간을 찾아서 별도의 컬럼('real_shot')에 새로 number 부여
# - 유효하지 않은 구간의 경우 'real_shot' 값을 -1로 설정
def find_and_mark_valid_shot(data, report_result=False):
    real_shot_no = 0
    
    data['real_shot'] = -1
    
    for shot in data['shot_no'].unique():
        real_shot_no += find_and_mark_valid_shot_of_specific_shot(data, shot, real_shot_no, report_result)
        #print(real_shot_no)

### 유효 shot 구간 내의 시작 부위의 유휴시간 데이터 삭제
 * 정상 공구 사용 패턴을 가지는 구간을 하나의 유효 shot으로 판별하고 별도의 column('real_shot')에 해당 numbering

In [388]:
#dataframe의 시작 부위의 idle section에 대한 'real_shot' 정보 초기화(값: -1)
# => 실제 가공이 이루어진 유효한 구간을 찾아냄
def remove_idle_section_at_the_start_of_valid_shot(data):
    #print(data)
    
    #스핀들 전류값이 0보다 크면 가공이 이루어진 구간으로 판단하여 아무 처리하지 않고 return
    if data.iloc[0]['current_spindle'] > 0:
        return
    
    last_index_of_idle_section = data[data['current_spindle'] > 0].index[0]-1
    
    print("index : {}~{}\n".format(data.index[0], last_index_of_idle_section))
    #print(data.loc[data.index[0]:last_index_of_idle_section])
    data.loc[data.index[0]:last_index_of_idle_section, 'real_shot'] = -1 #idle 구간의 valid shot 정보 초기화
    #print(data.loc[data.index[0]:last_index_of_idle_section])
    return

In [389]:
#dataframe의 유효한 shot 구간('real_shot' != -1인 구간) 별로 시작 부위의 idle section에 대한 'real_shot' 정보 초기화(값: -1)
# => 실제 가공이 이루어진 유효한 구간을 찾아냄
def remove_idle_sections_at_the_start_of_valid_shots(data):
    for shot in data['real_shot'].unique():
        if shot < 0:
            continue
            
#        df_by_shot = data[data['real_shot'] == shot]
        
#        if df_by_shot.iloc[0]['shot_no'] == 6517:
#            print(df_by_shot)
            
#        print(df_by_shot)
        index = data[data['real_shot'] == shot].index
    
        remove_idle_section_at_the_start_of_valid_shot(data.loc[index[0]:index[0]+len(index)-1])
        
        print(data.loc[index[0]:index[0]+len(index)-1])

    return

## 시각화

In [390]:
#dataframe에서 컬럼을 다중선택하여 데이터를 차트로 가시화
#① data : dataframe 입력
#② data_type_names : list 형태의 column명을 입력   ex) ['col1'], ['col1', 'col2'] 
#③ dataframe에서 특정 범위만을 선택해서 차트를 가시화 하려면 index_min 또는 index_max를 지정
#   .. index_min : 디폴트값(-1)인 경우에는 데이터의 시작 row부터 포함됨
#   ..  index_max : 디폴트값(-1)인 경우에는 데이터의 끝 row까지 포함됨
#④ 차트 제목 표시 관련 설정
#   .. title : 차트의 제목 문자열
#   .. title_font_size : 차트의 제목을 표시할 font size
#⑤ 차트 크기 설정
#   .. figsize_horz, figsize_vert : 차트의 가로, 세로 크기
#⑥ 범례 표시 관련 설정
#   .. legend_font_size : 범례를 표시할 font size
#   .. legend_location : 범례 표시 위치 ("upper right", "lower right", "upper left", "lower left")
#⑦ 그래프 표현 관련 설정 : 선 or marker 표시
#   .. linestyle : 선 스타일의 이름을 "solid", "dashed", "dotted", "dashdot"와 같은 형식으로 입력하거나 아래를 참고하여 입력
'''
'-'  solid line style
'--' line style
'-.' dash-dot line style
':'  dotted line style
'.'  point marker
','  pixel marker
'o'  circle marker
'v'  triangle_down marker
'^'  triangle_up marker
'<'  triangle_left marker
'>'  triangle_right marker
'1'  tri_down marker
'2'  tri_up marker
'3'  tri_left marker
'4'  tri_right marker
's'  square marker
'p'  pentagon marker
'*'  star marker
'h'  hexagon1 marker
'H'  hexagon2 marker
'+'  plus marker
'x'  x marker
'D'  diamond marker
'd'  thin_diamond marker
'|'  vline marker
'_'  hline marker
'''
#  .. marker : 아래를 참고하여 입력 ( https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.plot.html)
'''
'.'  point marker
','  pixel marker
'o'  circle marker
'v'  triangle_down marker
'^'  triangle_up marker
'<'  triangle_left marker
'>'  triangle_right marker
'1'  tri_down marker
'2'  tri_up marker
'3'  tri_left marker
'4'  tri_right marker
'8'  octagon marker
's'  square marker
'p'  pentagon marker
'P'  plus (filled) marker
'*'  star marker
'h'  hexagon1 marker
'H'  hexagon2 marker
'+'  plus marker
'x'  x marker
'X'  x (filled) marker
'D'  diamond marker
'd'  thin_diamond marker
'|'  vline marker
'_'  hline marker
'''
def show_plot(data, data_type_names, index_min = -1, index_max = -1, title = None, title_font_size=20,
              x_axis_title = None, y_axis_title = None, axes_title_font_size=18, tick_label_font_size=15,
              figsize_horz=20, figsize_vert=10, legend_font_size=15, legend_location="upper right",
              linestyle = 'none', marker='.', marker_size=10):
    
    parameters = { 'figure.titlesize':title_font_size, 'axes.titlesize': axes_title_font_size,
                  'axes.labelsize' : tick_label_font_size, 'legend.fontsize' : legend_font_size }
    plt.rcParams.update(parameters)
    
    plt.figure(figsize = (figsize_horz, figsize_vert))
    
    str_col_names = '{}'.format(','.join(data_type_names))#(f"'{col}'" for col in data_type_name))
    print(str_col_names)
    
    if index_min > -1 and index_max > -1 and index_min < index_max:
        for column in list(data_type_names):
            plt.plot(data.index[index_min:index_max], data.iloc[index_min:index_max][column], marker=marker, linestyle=linestyle, markersize=marker_size)
    elif index_min > -1:
        for column in list(data_type_names):
            plt.plot(data.index[index_min:], data.iloc[index_min:][column], marker=marker, linestyle=linestyle, markersize=marker_size)
    elif index_max > -1:
        for column in list(data_type_names):
            plt.plot(data.index[:index_max], data.iloc[:index_max][column], marker=marker, linestyle=linestyles, markersize=marker_size)
    else:
        #str_col_names = ','.join('{0}'.format(col) for col in data_type_name)
        for column in list(data_type_names):
            plt.plot(data.index, data[column], marker=marker, linestyle=linestyle, markersize=marker_size)
    
    if title is not None:
        plt.title(title)

    plt.legend(data_type_names, loc = legend_location)
    
    if x_axis_title is not None:
        plt.xlabel(x_axis_title)
    if y_axis_title is not None:
        plt.ylabel(y_axis_title)

In [391]:
#dataframe에서 컬럼을 다중선택하여 데이터를 차트로 가시화
#① data : dataframe 입력
#② data_type_name : 값을 관찰할 컬럼
#③ condition_column : 조건 판별 대상 컬럼명  ex) 'state'
#④ condition_values : 일치 여부 대상 value 목록  ex) [1], [0, 1]
#⑤ dataframe에서 특정 범위만을 선택해서 차트를 가시화 하려면 index_min 또는 index_max를 지정
#   .. index_min : 디폴트값(-1)인 경우에는 데이터의 시작 row부터 포함됨
#   ..  index_max : 디폴트값(-1)인 경우에는 데이터의 끝 row까지 포함됨
#⑥ 차트 제목 표시 관련 설정
#   .. title : 차트의 제목 문자열
#   .. title_font_size : 차트의 제목을 표시할 font size
#⑦ 차트 크기 설정
#   .. figsize_horz, figsize_vert : 차트의 가로, 세로 크기
#⑧ 범례 표시 관련 설정
#   .. legend_font_size : 범례를 표시할 font size
#   .. legend_location : 범례 표시 위치 ("upper right", "lower right", "upper left", "lower left")
#⑨ 그래프 표현 관련 설정 : 선 or marker 표시
#   .. linestyle : 선 스타일의 이름을 "solid", "dashed", "dotted", "dashdot"와 같은 형식으로 입력하거나 아래를 참고하여 입력
'''
'-'  solid line style
'--' line style
'-.' dash-dot line style
':'  dotted line style
'.'  point marker
','  pixel marker
'o'  circle marker
'v'  triangle_down marker
'^'  triangle_up marker
'<'  triangle_left marker
'>'  triangle_right marker
'1'  tri_down marker
'2'  tri_up marker
'3'  tri_left marker
'4'  tri_right marker
's'  square marker
'p'  pentagon marker
'*'  star marker
'h'  hexagon1 marker
'H'  hexagon2 marker
'+'  plus marker
'x'  x marker
'D'  diamond marker
'd'  thin_diamond marker
'|'  vline marker
'_'  hline marker
'''
#  .. marker : 아래를 참고하여 입력 ( https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.plot.html)
'''
'.'  point marker
','  pixel marker
'o'  circle marker
'v'  triangle_down marker
'^'  triangle_up marker
'<'  triangle_left marker
'>'  triangle_right marker
'1'  tri_down marker
'2'  tri_up marker
'3'  tri_left marker
'4'  tri_right marker
'8'  octagon marker
's'  square marker
'p'  pentagon marker
'P'  plus (filled) marker
'*'  star marker
'h'  hexagon1 marker
'H'  hexagon2 marker
'+'  plus marker
'x'  x marker
'X'  x (filled) marker
'D'  diamond marker
'd'  thin_diamond marker
'|'  vline marker
'_'  hline marker
'''
def show_plot_comparing_data_by_condition(data, data_type_name, condition_column, condition_values, data_names = ('condition is true','condition is false'),
                                           index_min = -1, index_max = -1, title = None, title_font_size=20,
                                           x_axis_title = None, y_axis_title = None, axes_title_font_size=18,
                                           tick_label_font_size=15, figsize_horz=20, figsize_vert=10,
                                           legend_font_size=15, legend_location="upper right", linestyle = 'none',
                                           marker='.', marker_size=10):
    
    parameters = { 'figure.titlesize':title_font_size, 'axes.titlesize': axes_title_font_size,
                  'axes.labelsize' : tick_label_font_size, 'legend.fontsize' : legend_font_size }
    plt.rcParams.update(parameters)
    
    plt.figure(figsize = (figsize_horz, figsize_vert))
    
    condition_true_data = pd.DataFrame()
    condition_false_data = pd.DataFrame()
      
    if index_min > -1 and index_max > -1 and index_min < index_max:
        condition_true_data = data.iloc[index_min:index_max].query(data[condition_column].isin(condition_values))
        condition_false_data = data.iloc[index_min:index_max].query(~data[condition_column].isin(condition_values))
    elif index_min > -1:
        condition_true_data = data.iloc[index_min:].query(data[condition_column].isin(condition_values))
        condition_false_data = data.iloc[index_min:].query(~data[condition_column].isin(condition_values))        
    elif index_max > -1:
        condition_true_data = data.iloc[:index_max].query(data[condition_column].isin(condition_values))
        condition_false_data = data.iloc[:index_max].query(~data[condition_column].isin(condition_values))        
    else:
        condition_true_data = data[data[condition_column].isin(condition_values)]
        condition_false_data = data[~data[condition_column].isin(condition_values)]
        
    plt.plot(condition_true_data.index, condition_true_data[data_type_name], marker=marker, linestyle=linestyle, label=data_names[0], color='red', markersize=marker_size)
    plt.plot(condition_false_data.index, condition_false_data[data_type_name], marker=marker, linestyle=linestyle, label=data_names[1], markersize=marker_size)
    
    if title is not None:
        plt.title(title)

    plt.legend(data_names, loc = legend_location)
    
    if x_axis_title is not None:
        plt.xlabel(x_axis_title)
    if y_axis_title is not None:
        plt.ylabel(y_axis_title)

## 모델 수립

### 유효한 shot을 기준으로 주어진 공구에 대한 주어진 field의 대푯값을 계산하여 dataframe 구성

In [392]:
def delete_outliers(data, column):
    Q1 = np.percentile(data[column], 25)
    Q3 = np.percentile(data[column], 75)
    IQR = Q3 - Q1

    outlier_step = 1.5 * IQR
    
    print('Q1 = {}, Q3 = {}, IQR = {}, outlier_step = {}, range = ({}. {})'.format(Q1, Q3, IQR, outlier_step, Q1-outlier_step, Q3+outlier_step))

    outlier_index = data[(data[column] < Q1) | (data[column] > Q3 + outlier_step)].index
        
    print(outlier_index)
    
    data.drop(outlier_index, inplace=True)

In [393]:
# 유효한 shot을 기준으로 주어진 공구에 대한 주어진 field의 대푯값을 계산하여  dataframe으로 구성
def build_representative_data_for_tool_machining_org(data, tool_no, data_type_name, apply_robust_scaler=False, except_min_value=False, except_max_value=False, delete_outlier=False,
                                                 delete_zero_value=False, except_tool_cancel_state=False, except_tool_end_state=False):
    if data_type_name not in data.columns:
        print("{} is not in columns\nPlease input valid column name".format(data_type_name))
        return pd.DataFrame()
    
    condition = (data['tool']==tool_no) & (data['real_shot'] != -1)
    
    #LJY_20221005 : tool취소 상태 데이터 제거 옵션 추가
    if except_tool_cancel_state is True:
        condition = condition & (data['tool_state'] != 0)
        
    if except_tool_end_state is True:
        condition = condition & (data['tool_state'] != 9)
        
    #df_tool = data[(data['tool']==tool_no) & (data['real_shot'] != -1)]
    df_tool = data[condition]
    if len(df_tool) < 1:
        return pd.DataFrame()
    
    if delete_zero_value is True:
        df_tool.drop(df_tool[df_tool[data_type_name] == 0].index, inplace=True)
        
    if delete_outlier is True:
        print('before deleting outliers : {}'.format(len(df_tool.index)))
        delete_outliers(df_tool, data_type_name)
        print('after deleting outliers : {}'.format(len(df_tool.index)))
    
    if except_min_value is True:
        min_value = df_tool[data_type_name].min()
        df_tool.drop(df_tool[df_tool[data_type_name] == min_value].index, inplace=True)
        
    if except_max_value is True:
        max_value = df_tool[data_type_name].max()
        df_tool.drop(df_tool[df_tool[data_type_name] == max_value].index, inplace=True)
    
    if apply_robust_scaler is True:
        from sklearn import metrics
        #from sklearn.preprocessing import MinMaxScaler
        #from sklearn.preprocessing import StandardScaler
        from sklearn.preprocessing import RobustScaler
        
        rs = RobustScaler()
        #ms = MinMaxScaler()
        df_tool[[data_type_name]] = rs.fit_transform(df_tool[[data_type_name]])
    
    #real shot no.
#    df_shot_no = df_tool['real_shot'].to_frame(name='real_shot')
    group_by_shot = df_tool.groupby(['real_shot'])
    #print(group_by_shot.groups.keys())
    
    #표준편차
    df_std = (group_by_shot.std()[data_type_name].to_frame(name='std'))
    #표준오차
    df_sem = (group_by_shot.sem()[data_type_name].to_frame(name='sem'))
    #합
    df_sum = (group_by_shot.sum()[data_type_name].to_frame(name='sum'))
    #평균
    df_mean = (group_by_shot.mean()[data_type_name].to_frame(name='mean'))
    #중앙값
    df_median = (group_by_shot.median()[data_type_name].to_frame(name='median'))
    #분산
    df_var = (group_by_shot.var()[data_type_name].to_frame(name='var'))
    #최대값
    df_max = (group_by_shot.max()[data_type_name].to_frame(name='max')) 
    #0.25분위수
    df_quantile_1_per_4 = (group_by_shot.quantile(0.25)[data_type_name].to_frame(name='quantile(0.25)')) 
    #0.75분위수
    df_quantile_3_per_4 = (group_by_shot.quantile(0.75)[data_type_name].to_frame(name='quantile(0.75)')) 
    #왜도
    df_skew = (group_by_shot.skew()[data_type_name].to_frame(name='skew')) 

    df_representative = pd.concat([df_mean, df_std, df_median, df_max, df_var, df_sem, df_sum, df_quantile_1_per_4, df_quantile_3_per_4, df_skew],axis=1)
    
    df_representative.set_index(pd.Series(group_by_shot.groups.keys()))
    
    return df_representative

In [394]:
# 유효한 shot을 기준으로 주어진 공구에 대한 주어진 field의 대푯값을 계산하여  dataframe으로 구성
def build_representative_data_for_tool_machining(data, tool_no, data_type_name, machine_id=22, apply_robust_scaler=False, except_min_value=False, except_max_value=False, delete_outlier=False,
                                                 delete_zero_value=False, except_tool_cancel_state=False, except_tool_end_state=False):
    if data_type_name not in data.columns:
        print("{} is not in columns\nPlease input valid column name".format(data_type_name))
        return pd.DataFrame()
    
    condition = (data['tool']==tool_no) & (data['real_shot'] != -1)
    
    #LJY_20221005 : tool취소 상태 데이터 제거 옵션 추가
    if except_tool_cancel_state is True:
        condition = condition & (data['tool_state'] != 0)
        
    if except_tool_end_state is True:
        condition = condition & (data['tool_state'] != 9)
        
    #df_tool = data[(data['tool']==tool_no) & (data['real_shot'] != -1)]
    df_tool = data[condition]
    if len(df_tool) < 1:
        return pd.DataFrame()
    
    if delete_zero_value is True:
        df_tool.drop(df_tool[df_tool[data_type_name] == 0].index, inplace=True)
        
    if delete_outlier is True:
        print('before deleting outliers : {}'.format(len(df_tool.index)))
        delete_outliers(df_tool, data_type_name)
        print('after deleting outliers : {}'.format(len(df_tool.index)))
    
    if except_min_value is True:
        min_value = df_tool[data_type_name].min()
        df_tool.drop(df_tool[df_tool[data_type_name] == min_value].index, inplace=True)
        
    if except_max_value is True:
        max_value = df_tool[data_type_name].max()
        df_tool.drop(df_tool[df_tool[data_type_name] == max_value].index, inplace=True)
    
    if apply_robust_scaler is True:
        from sklearn import metrics
        #from sklearn.preprocessing import MinMaxScaler
        #from sklearn.preprocessing import StandardScaler
        from sklearn.preprocessing import RobustScaler
        
        rs = RobustScaler()
        #ms = MinMaxScaler()
        df_tool[[data_type_name]] = rs.fit_transform(df_tool[[data_type_name]])
    
    #real shot no.
#    df_shot_no = df_tool['real_shot'].to_frame(name='real_shot')
    group_by_shot = df_tool.groupby(['real_shot'])
    #print(group_by_shot.groups.keys())
    
    #표준편차
    df_std = (group_by_shot.std()[data_type_name].to_frame(name='data_std'))
    #표준오차
    df_sem = (group_by_shot.sem()[data_type_name].to_frame(name='data_sem'))
    #합
    df_sum = (group_by_shot.sum()[data_type_name].to_frame(name='data_sum'))
    #평균
    df_mean = (group_by_shot.mean()[data_type_name].to_frame(name='data_mean'))
    #중앙값
    df_median = (group_by_shot.median()[data_type_name].to_frame(name='data_median'))
    #분산
    df_var = (group_by_shot.var()[data_type_name].to_frame(name='data_var'))
    #최대값
    df_max = (group_by_shot.max()[data_type_name].to_frame(name='data_max')) 
    #0.25분위수
    df_quantile_1_per_4 = (group_by_shot.quantile(0.25)[data_type_name].to_frame(name='data_quantitle_1_per_4')) 
    #0.75분위수
    df_quantile_3_per_4 = (group_by_shot.quantile(0.75)[data_type_name].to_frame(name='data_quantitle_3_per_4')) 
    #왜도
    df_skew = (group_by_shot.skew()[data_type_name].to_frame(name='data_skew')) 

    df_representative = pd.concat([df_mean, df_std, df_median, df_max, df_var, df_sem, df_sum, df_quantile_1_per_4, df_quantile_3_per_4, df_skew],axis=1)
    
    df_representative['date'] = group_by_shot.first()['date']
    df_representative['datetime'] = group_by_shot.first()['datetime']
    df_representative['shot_no'] = group_by_shot.first()['shot_no']
    df_representative['shot_no_sub'] = group_by_shot.groups.keys()
    df_representative['mc_id'] = machine_id
    df_representative['tool_number'] = tool_no
    #df_representative.rename(columns = {'real_shot' : 'shot_no_sub'}, inplace = True)
    #df_representative.set_index(pd.Series(group_by_shot.groups.keys()))
    
    df_representative.reset_index(drop=True,inplace=True)
    
    change_data_type_64bit_to_32bit(df_representative) #데이터 타입 32bit로 변환
    
    return df_representative

### 주어진 공구의 주어진 필드에 대한 valid shot 별 대푯값 도출 & 시각화

In [395]:
#representative_value_types : 'std', 'sem', 'sum', 'mean', 'median', 'var', 'max', 'quantile(0.25)', 'quantile(0.75)', 'skew'를 리스트 형태로 입력
#  ex) ['std', 'mean']
def build_and_display_representative_data_for_tool_machining(data, tool_no, data_type_name, representative_value_types, apply_robust_scaler=False, except_min_value=False, except_max_value=False):
    df_representative = build_representative_data_for_tool_machining(data, tool_no, data_type_name, apply_robust_scaler, except_min_value, except_max_value)
    show_plot(df_representative, representative_value_types, title = data_type_name, x_axis_title='real shot no.', y_axis_title='representatives')

## 이상탐지

### IsolationForest 활용

In [396]:
#dataframe을 입력하여 model로부터 threshold 값 계산
def run_anomaly_detaction_by_IsolationForest_org(model, data, stat_type_name='data_mean'):
    anomaly_score = model.decision_function(data[[stat_type_name]])
    outlier = model.predict(data[[stat_type_name]])
    data['outliers'] = pd.Series(model.predict(data[[stat_type_name]])).apply(lambda x: 1 if x == -1 else 0)
    data['anomaly_score'] = anomaly_score

In [397]:
#dataframe을 입력하여 model로부터 threshold 값 계산
def run_anomaly_detaction_by_IsolationForest(model, data, stat_type_name='data_mean'):
    anomaly_score = model.decision_function(data[[stat_type_name]])
    outlier = model.predict(data[[stat_type_name]])
    data['outliers'] = pd.Series(model.predict(data[[stat_type_name]])).apply(lambda x: 1 if x == -1 else 0)
    data['anomaly_score'] = anomaly_score

In [398]:
#dataframe을 입력하여 model로부터 threshold 값 계산
def calculate_threshold_value_by_IsolationForest(model, data, stat_type_name='data_mean'):
    run_anomaly_detaction_by_IsolationForest(model, data, stat_type_name)
    
    #value_range = data[stat_type_name].max() - data[stat_type_name].min()

    threshold_value = min(data.query('outliers == 1')[stat_type_name])# + value_range * model.get_params()['contamination']

    return threshold_value

In [399]:
# he offsetis defined in such a way we obtain the expected number of outliers (samples with decision function < 0) in training.
def update_IsolationForest_model_for_anomaly_detaction(data, machine_id=22, tool_no=5, data_type_name='current_spindle', stat_type_name='data_mean', save_model=False):
    model = IsolationForest(random_state = 42, contamination = 0.004 , n_estimators = 500, max_samples = 90, n_jobs = -1, bootstrap = True)
    model.fit(data[[stat_type_name]])
    
    calculate_threshold_value_by_IsolationForest(model, data, stat_name)
        
    #threshold값을 DB에 저장
    put_threshold_data_from_db_by_data_type_name(engine, threshold_value, machine_id=machine_id, tool_no=tool_no, data_type_name=data_type_name, stat_type_name=stat_type_name)
   
    if save_model is False:
        return model
    
    import pickle
    import joblib
    model_path = make_valid_model_file_path(engine, model_name='Isolation Forest', machine_id=machine_id, tool_no=tool_no, data_type_name=data_type_name, stat_type_name=stat_type_name)
    
    if len(model_path) < 1:
        return model
    
    joblib.dump(model, model_path)
    return model

## DB 연동 처리 (sqlalchemy 활용)
  * DB에서 데이터 불러오기 테스트 결과 가장 효율적인 모듈인 sqlalchemy 활용

### DB 연결

In [400]:
#DB Connection
def connect_db(dbIP = '112.220.81.163', dbID = 'DEASHIN_SIGNAL', dbPW = 'DEASHIN_SIGNAL', dbPort = 9933, dbName = 'DAESHIN_CLIVE'):
    engine = create_engine("mysql://{user}:{pw}@{host}:{port}/{db}".format(user=dbID, pw=dbPW, host=dbIP, port=dbPort, db=dbName))
        
    return engine

### 설비 정보 조회

In [401]:
#설비 이름을 반환
def get_machine_name_from_db(engine, machine_id=22):
    table = pd.read_sql_query("SELECT * FROM tbMachine WHERE n_MC_ID = {0}".format(machine_id), engine)
    
    machine_name = ""
    
    if len(table) > 0:
        machine_name = table.iloc[0]['s_MC_No']
        del table
        gc.collect()
    
    return machine_name

### mc_id(설비 ID)가 주어진 machine_id인 shot_no list 추출

#### 주어진 shot_no가 주어진 machine의 데이터인지 체크

In [402]:
#주어진 shot_no의 데이터의 machine id가 machine_id와 동일한지 DB를 이용하여 체크
def check_machine_id_for_shot_no(engine, shot_no, machine_id=22, report_result=False):
    table = pd.read_sql_query('SELECT mc_id FROM TPOP_MACHINE_PARAMETER where shot_no = {0} LIMIT 1'.format(shot_no), engine)
    if len(table) < 1:
        return False
    
    shot_mc_id = table.iloc[0]['mc_id']
    
    del table
    gc.collect()
    
    if shot_mc_id != machine_id:
        return False
    
    if report_result is True:
        print('machine id of shot({0}) : {1}'.format(shot_no, shot_mc_id))
        
    return True

#### 설비 신호 데이터 테이블(TPOP_MACHINE_PARAMETER)에서 shot_no list 추출

In [403]:
#DB의 TPOP_MACHINE_PARAMETER 테이블에 저장된 shot_no list 추출
def get_shot_no_list_from_db(engine):
    table = pd.read_sql_query('SELECT DISTINCT shot_no FROM TPOP_MACHINE_PARAMETER ORDER BY shot_no', engine)
    table=table.dropna(subset=['shot_no'])
    index = table[table['shot_no'] == 0].index
    table.drop(index , inplace=True)
    return table['shot_no'].to_list()

#### 설비 신호 데이터 테이블(TPOP_MACHINE_PARAMETER)에서 machine_id가 일치하는 shot_no list 추출

In [404]:
#DB의 TPOP_MACHINE_PARAMETER 테이블에 저장된 shot_no list 추출
def get_shot_no_list_by_machine_id_from_db(engine, machine_id=22, report_result=False):
    #TPOP_MACHINE_PARAMETER 테이블의 shot_no list 전체 가져오기
    all_shot_no = get_shot_no_list_from_db(engine)

    if len(all_shot_no) < 1:
        return all_shot_no

    shot_no_by_machine_id = []
    res = False
    
    count = 0
    
    for shot_no in all_shot_no:
        #shot_no에 해당하는 shot의 machine id가 주어진 machine_id와 일치하는지 체크
        res = check_machine_id_for_shot_no(engine, shot_no, machine_id=machine_id, report_result=report_result)

        #shot의 mc_id가 machine_id와 일치하는 경우에만 별도의 list에 추가
        if res is True:
            shot_no_by_machine_id.append(shot_no)
        
        print("loop {0}, shot_no {1} # of shot = {2}".format(count, shot_no, len(shot_no_by_machine_id)))
        count += 1
    
    print(shot_no_by_machine_id)#temp
    all_shot_no.clear()
    
    #주어진 machine_id에 해당하는 shot_no list 반환
    return shot_no_by_machine_id

### 유효 shot 도출 & 대푯값 산출한 마지막 shot 이후의 shot_no 목록 추출
  * 용도 : 마지막으로 저장된 shot 이후의 데이터에 대해서 대푯값을 추출하여 저장하기 위함
  * 특정 시점에 기능을 실행했을 때 최신 데이터에 대해 유효 shot 구간 도출 & 대푯값 산출 결과를 저장할 수 있도록 함

#### 대푯값 저장 table(tbShotStatistics)에서 mc_id(설비 ID), tool_number(공구 번호) 기준으로 마지막 shot 정보 가져오기

In [405]:
def get_last_shot_info_at_shot_statistics_db(engine, machine_id=22, tool_no=5):
    table = pd.read_sql_query('SELECT datetime, shot_no, shot_no_sub, mc_id, tool_number\
                            FROM tbShotStatistics where mc_id={0} and tool_number={1} ORDER BY\
                            shot_no DESC, shot_no_sub DESC LIMIT 1'.format(machine_id, tool_no), engine)
    return table

#### 설비 신호 데이터 테이블(TPOP_MACHINE_PARAMETER)에서 주어진 시간, shot_no 이후의 shot_no list 추출

In [406]:
#DB의 TPOP_MACHINE_PARAMETER 테이블에 저장된 shot_no list 추출
def get_shot_no_list_from_db_after_given_datetime_and_shot_no(engine, datetime, prev_shot_no, machine_id=22):
    table = pd.read_sql_query("SELECT DISTINCT shot_no FROM TPOP_MACHINE_PARAMETER WHERE datetime > '{0}' AND mc_id = {1} AND shot_no > {2}".format(datetime, machine_id, prev_shot_no), engine)
    return table['shot_no'].to_list()

### 설비 신호 데이터 테이블(TPOP_MACHINE_PARAMETER)에서  주어진 shot_no, machine_id에 해당하는 데이터 추출

In [407]:
def get_shot_data_from_db(engine, shot_no, columns, machine_id=22, report_result=False):
    strcolumn = ', '.join(map(str, columns))

    query = "SELECT date, datetime, shot_no, tool_number, {0} from TPOP_MACHINE_PARAMETER WHERE shot_no = {1} and mc_id = {2}".format(strcolumn, shot_no, machine_id)
    table = pd.read_sql_query(query, engine)
    
    if len(table) < 1:
        return table

    table = table.drop(table[table['tool_number'] == -1].index)
    change_data_type_64bit_to_32bit(table)
    
    extract_tool_info_from_data(table)
    
    if report_result is True:
        print("(1) shot_no : {0}\n (2) machine_id : {1} \n(3) data \n".format(shot_no, machine_id))
        print("----- dataframe info.\n{}\n\n----- data\n",format(table.info()))
        print(table)
    
    return table

### tbShotStatistics 테이블에서 주어진 machine_id, tool_no에 대한 데이터를 추출하여 dataframe 구성

In [408]:
#data_type : 대표값 데이터 타입 (current_spindle : 0, current_x = 1, current_z = 2)
def get_all_shot_statistics_data_from_db(engine, machine_id=22, tool_no=5, data_type=0, report_result=False):
    import pandas as pd

    query = "SELECT * from tbShotStatistics WHERE mc_id = {0} and tool_number = {1} and data_type = {2} ORDER BY datetime".format(machine_id, tool_no, data_type)
    
    if report_result is True:
        print(query)
        
    table = pd.read_sql_query(query, engine)#, index_col='datetime')
    
    table.reset_index()
    
    if report_result is True:
        print(table)
    
    return table

In [409]:
#data_type : 대표값 데이터 타입 (current_spindle : 0, current_x = 1, current_z = 2)
def get_all_shot_statistics_data_from_db_by_data_type_name(engine, machine_id=22, tool_no=5, data_type_name = 'current_spindle', report_result=False):
    data_type = 0
    
    if data_type_name == 'current_spindle':
        data_type = 0
    elif data_type_name == 'current_x':
        data_type = 1
    elif data_type_name == 'current_z':
        data_type = 2
    else:
        return pd.DataFrame()
    
    return get_all_shot_statistics_data_from_db(engine, machine_id, tool_no, data_type, report_result)

In [410]:
#data_type : 대표값 데이터 타입 (current_spindle : 0, current_x = 1, current_z = 2)
def get_all_shot_specific_statistic_data_from_db(engine, machine_id=22, tool_no=5, data_type=0, stat_type_name ='data_mean', report_result=False):
    import pandas as pd

    query = "SELECT date, datetime, mc_id, tool_number, shot_no, shot_no_sub, data_type, {0} from tbShotStatistics WHERE mc_id = {1} and tool_number = {2} and data_type = {3} ORDER BY datetime".format(stat_type_name, machine_id, tool_no, data_type)
    
    if report_result is True:
        print(query)
        
    table = pd.read_sql_query(query, engine)#, index_col='datetime')
    
    table.reset_index()
    
    if report_result is True:
        print(table)
    
    return table

In [411]:
#data_type : 대표값 데이터 타입 (current_spindle : 0, current_x = 1, current_z = 2)
def get_all_shot_specific_statistic_data_from_db_by_data_type_name(engine, machine_id=22, tool_no=5, data_type_name = 'current_spindle',
                                                                stat_type_name ='data_mean', report_result=False):
    data_type = 0
    
    if data_type_name == 'current_spindle':
        data_type = 0
    elif data_type_name == 'current_x':
        data_type = 1
    elif data_type_name == 'current_z':
        data_type = 2
    else:
        return pd.DataFrame()
    
    return get_all_shot_specific_statistic_data_from_db(engine, machine_id, tool_no, data_type, stat_type_name, report_result)

### valid shot 별로 대푯값을 추출하여 tbShotStatistics 테이블 업데이트

In [412]:
def update_shot_statistics_db(engine, df_shot, tool_no, column, machine_id=22):
    #유효shot 별로 대푯값 추출
    df_representative = build_representative_data_for_tool_machining(df_shot, tool_no, column, apply_robust_scaler=False, except_min_value=False, except_max_value=False, except_tool_cancel_state=False, except_tool_end_state=False)
    
    if len(df_representative) < 1:
        return
    
    #데이터 타입 판별 (우선 3개의 필드에 대해서만 판별)
    if column == "current_spindle":
        df_representative['data_type'] = 0
    elif column == "current_x":
        df_representative['data_type'] = 1
    elif column == "current_z":
        df_representative['data_type'] = 2
    
    #DB에 데이터 추가
    df_representative.to_sql('tbShotStatistics', con=engine, if_exists='append', index=False)
    
    del df_representative
    gc.collect()

### 주어진 shot에 대해 유효 shot 구간 도출 및 대푯값 추출 후 tbShotResult 테이블 업데이트

In [413]:
def extract_valid_shot_and_update_shot_statistics_db(engine, shot_no, tool_no, columns, machine_id=22):
    #shot_no(shot 번호), mc_id(설비 ID)에 해당하는 데이터 추출
    df_shot = get_shot_data_from_db(engine, shot_no, columns, machine_id=machine_id, report_result=False)
    
    #유효 shot 구간 도출 및 numbering 처리 ("real_shot" 필드에 값 저장)
    find_and_mark_valid_shot(df_shot, report_result=False)
    
    #주어진 column들에 대해 유효 shot 구간 별로 대푯값을 추출하여 tbShotResult 업데이트 
    for column in columns:
        update_shot_statistics_db(engine, df_shot, tool_no, column, machine_id=machine_id)
    
    del df_shot
    gc.collect()

### 주어진 machine_id에 해당하는 설비의 전체 shot에 대한 (1) 유효 shot 구간 도출, (2) 유효 shot 별 대푯값 추출, (3) shot 결과 테이블(tbShotResult) 업데이트

In [414]:
def extract_valid_shot_and_update_shot_statistics_db_for_all_shot(engine, columns, machine_id=22, tool_no=5, report_result=False):
    #TPOP_MACHINE_PARAMETER 테이블의 shot_no list 전체 가져오기
    all_shot_no = get_shot_no_list_from_db(engine)

    if len(all_shot_no) < 1:
        return all_shot_no

    res = False
    count = 0

    for shot_no in all_shot_no:
        #shot_no에 해당하는 shot의 machine id가 주어진 machine_id와 일치하는지 체크
        res = check_machine_id_for_shot_no(engine, shot_no, machine_id=machine_id, report_result=report_result)

        #shot의 mc_id가 machine_id와 일치하는 경우에만 별도의 list에 추가
        if res is True:
            print("loop {0}, shot_no {1} extract".format(count, shot_no))
            extract_valid_shot_and_update_shot_statistics_db(engine, shot_no, tool_no, columns)
            
        print("loop {0}, shot_no {1}".format(count, shot_no))
        count += 1

In [415]:
def extract_valid_shot_and_update_shot_statistics_db_for_all_shot_after_given_shot_old(engine, prev_shot_datetime, prev_shot_no, columns, machine_id=22, tool_no=5, report_result=False):
    #TPOP_MACHINE_PARAMETER 테이블의 shot_no list 전체 가져오기
    all_shot_no = get_shot_no_list_from_db(engine)

    if len(all_shot_no) < 1:
        return all_shot_no

    res = False
    count = 0
    
    all_shot_no.clear()
    all_shot_no = get_shot_no_list_from_db_after_given_datetime_and_shot_no(engine, prev_shot_datetime,
                                                                              prev_shot_no, machine_id)
    
    for shot_no in all_shot_no:
        if shot_no <= prev_shot_no:
            continue
            
        #shot_no에 해당하는 shot의 machine id가 주어진 machine_id와 일치하는지 체크
        res = check_machine_id_for_shot_no(engine, shot_no, machine_id=machine_id, report_result=report_result)

        #shot의 mc_id가 machine_id와 일치하는 경우에만 별도의 list에 추가
        if res is True:
            print("loop {0}, shot_no {1} extract".format(count, shot_no))
            extract_valid_shot_and_update_shot_statistics_db(engine, shot_no, tool_no, columns)
            
        print("loop {0}, shot_no {1}".format(count, shot_no))
        count += 1

In [416]:
def extract_valid_shot_and_update_shot_statistics_db_for_all_shot_after_given_shot(engine, prev_shot_datetime, prev_shot_no, columns, machine_id=22, tool_no=5, report_result=False):
    all_shot_no = get_shot_no_list_from_db_after_given_datetime_and_shot_no(engine, prev_shot_datetime,
                                                                              prev_shot_no, machine_id)

    if len(all_shot_no) < 1:
        return

    count = 0
    
    for shot_no in all_shot_no:
        print("loop {0}, shot_no {1} extract".format(count, shot_no))
        extract_valid_shot_and_update_shot_statistics_db(engine, shot_no, tool_no, columns)
        count += 1

In [417]:
def extract_and_update_representatives_for_all_shot_with_db(columns, report_result=False, machine_id=22, tool_no=5):
    #DB 연결
    engine = connect_db()

    #LJY_20221022 : tbShotResult에 저장된 마지막 shot 정보를 가져오기
    df_last_shot = get_last_shot_info_at_shot_statistics_db(engine, machine_id=machine_id, tool_no=tool_no)
    
    res = False

    #shot 대푯값 추출 결과 테이블(tbShotResult)가 비어있는 경우 TPOP_MACHINE_PARAMETER 테이블의 전체 데이터 중
    #  machine_id가 일치하는 shot list 추출
    if len(df_last_shot) < 1:
        if report_result is True:
            print("Need to extract representative values for all shots with machine id {0}, tool_no {1}".format(machine_id, tool_no))
        
        extract_valid_shot_and_update_shot_statistics_db_for_all_shot(engine, machine_id, tool_no, columns)
        #shot_list = get_shot_no_list_by_machine_id_from_db(engine, machine_id=machine_id)
        engine.dispose()
        return

    if report_result is True:
        print("Last shot from shot results table(tbShotResult) is -----\n{0}", df_last_shot.to_string())

    print(df_last_shot)#temp
    
    extract_valid_shot_and_update_shot_statistics_db_for_all_shot_after_given_shot(engine, df_last_shot.iloc[0]['datetime'], df_last_shot.iloc[0]['shot_no'], columns, machine_id, tool_no)

    del df_last_shot
    gc.collect()
    
    engine.dispose()

### 주어진 mc_id(설비 ID),  tool_no(공구 번호), data_type(대푯값 데이터 타입)에 대한 threshold 조회/저장
  * data_type : 대표값 데이터 타입 (current_spindle : 0, current_x = 1, current_z = 2)

In [418]:
#threshold DB에서 마지막 데이터를 반환
def get_threshold_data_from_db(engine, machine_id=22, tool_no=5, data_type=0, stat_type_name='data_mean'):
    query = "SELECT * FROM tbShotThreshold WHERE mc_id = {0} AND tool_number = {1} AND data_type = {2}\
                    AND stat_type = '{3}' ORDER BY datetime DESC LIMIT 1".format(machine_id, tool_no, data_type, stat_type_name)
    table = pd.read_sql_query(query, engine)
    print(table)
    return table

In [419]:
def get_threshold_data_from_db_by_data_type_name(engine, machine_id=22, tool_no=5, data_type_name='current_spindle', stat_type_name='data_mean'):
    data_type = 0
    
    if data_type_name == 'current_spindle':
        data_type = 0
    elif data_type_name == 'current_x':
        data_type = 1
    elif data_type_name == 'current_z':
        data_type = 2
    else:
        return pd.DataFrame()
    
    return get_threshold_data_from_db(engine, machine_id, tool_no, data_type, stat_type_name)

In [420]:
#threshold DB에 값 저장
def put_threshold_data_from_db(engine, threshold, machine_id=22, tool_no=5, data_type=0, stat_type_name='data_mean'):
    from datetime import datetime
    
    now = datetime.now()
    
    str_date = now.strftime("%Y-%m-%d")
    str_datetime = now.strftime("%Y-%m-%d %H:%M:%S")
    
    query = "INSERT INTO tbShotThreshold (date, datetime, mc_id, tool_number, data_type, stat_type, threshold) VALUES\
            ('{0}', '{1}', {2}, {3}, {4}, '{5}', {6})".format(str_date, str_datetime, machine_id, tool_no, data_type, stat_type_name, threshold)
    
    engine.execute(query)
    
    return True

In [421]:
def put_threshold_data_from_db_by_data_type_name(engine, threshold, machine_id=22, tool_no=5, data_type_name='current_spindle', stat_type_name='data_mean'):
    data_type = 0
    
    if data_type_name == 'current_spindle':
        data_type = 0
    elif data_type_name == 'current_x':
        data_type = 1
    elif data_type_name == 'current_z':
        data_type = 2
    else:
        return False
    
    return put_threshold_data_from_db(engine, threshold, machine_id, tool_no, data_type, stat_type_name)

In [422]:
'''
engine = connect_db()
path = make_valid_model_file_path(engine)
model = joblib.load(path)
#print(model.get_params())
print(model.feature_names_in_)
print(model.offset_)
engine.dispose()
'''

'\nengine = connect_db()\npath = make_valid_model_file_path(engine)\nmodel = joblib.load(path)\n#print(model.get_params())\nprint(model.feature_names_in_)\nprint(model.offset_)\nengine.dispose()\n'

In [423]:
def get_threshold_value_by_data_type_name(engine, machine_id=22, tool_no=5, data_type_name ='current_spindle', stat_type_name='data_mean', report_result=False):
    df_threshold = get_threshold_data_from_db_by_data_type_name(engine, machine_id, tool_no, data_type_name, stat_type_name)
    
    threshold_value = 0
    
    #threshold 정보가 DB에 있는 경우
    if len(df_threshold) > 0:
        threshold_value = df_threshold.iloc[0]['threshold']
        del df_threshold
        gc.collect()
        return threshold_value

    model_path = make_valid_model_file_path(engine, model_name='Isolation Forest', machine_id=machine_id, tool_no=tool_no, data_type_name=data_type_name, stat_type_name=stat_type_name)
    
    print(model_path)
    if len(model_path) < 1:
        return np.NaN
    
    df_shot_statistics = get_all_shot_specific_statistic_data_from_db_by_data_type_name(engine, machine_id, tool_no, data_type_name, stat_type_name)
    if len(df_shot_statistics) < 1:
        return np.NaN
    
    if report_result is True:
        print(df_shot_statistics.info())
    
    model = joblib.load(model_path)
    #model로부터 threshold 값 계산
    threshold_value = calculate_threshold_value_by_IsolationForest(model, df_shot_statistics, stat_type_name=stat_type_name)
    
    #threshold값을 DB에 저장
    put_threshold_data_from_db_by_data_type_name(engine, threshold_value, machine_id=machine_id, tool_no=tool_no,
                                                 data_type_name=data_type_name, stat_type_name=stat_type_name)
    
    if report_result is True:
        print(threshold_value)
        print(df_shot_statistics[df_shot_statistics['outliers'] == 1])

        show_plot_comparing_data_by_condition(df_shot_statistics, 'data_mean', 'outliers', [1], data_names = ('outlier', 'normal'),
                                          title = 'Spindle Current', x_axis_title='real shot no.', y_axis_title='current_spindle')
    
    return threshold_value

In [424]:
'''
engine = connect_db()
print(get_threshold_value_by_data_type_name(engine, report_result=False))
engine.dispose()
'''

'\nengine = connect_db()\nprint(get_threshold_value_by_data_type_name(engine, report_result=False))\nengine.dispose()\n'

### anomaly detaction(이상탐지)를 실행하여 결과를 저장한 마지막 shot 이후의 데이터 추출
  * 용도 : 마지막으로 이상탐지 결과를 저장한 shot 이후의 데이터에 대해서 이상탐지를 수행한 후 결과를 저장하기 위함
  * 특정 시점에 기능을 실행했을 때 최신 데이터에 대해 이상탐지 결과를 저장할 수 있도록 함

#### 이상탐지 결과 저장 table(tbShotResult)에서 mc_id(설비 ID), tool_number(공구 번호) 기준으로 마지막 shot 정보 가져오기

In [425]:
def get_last_shot_info_at_shot_result_db(engine, machine_id=22, tool_no=5):
    query = "SELECT * FROM tbShotResult where mc_id={0} and tool_number={1} ORDER BY\
            shot_no DESC, shot_no_sub DESC LIMIT 1".format(machine_id, tool_no)
    table = pd.read_sql_query(query, engine)
    return table

#### shot 별 통계 데이터 테이블(tbShotStatistics)에서 주어진 시간, shot_no 이후의 data 추출

In [426]:
#DB의 tbShotStatistics 테이블에 저장된 shot_no list 추출
def get_all_shot_specific_stat_data_after_given_prev_data(engine, prev_datetime, prev_shot_no, machine_id=22, tool_no=5, data_type=0, stat_type_name='data_mean'):
    query = "SELECT date, datetime, mc_id, tool_number, shot_no, shot_no_sub, data_type, {0} FROM tbShotStatistics WHERE shot_no > {1} AND datetime > '{2}' AND mc_id = {3}\
                                AND tool_number = {4} AND data_type = {5}".format(stat_type_name, prev_shot_no, prev_datetime, machine_id, tool_no, data_type)
    table = pd.read_sql_query(query, engine)
    return table

### shot 별 이상 판단 결과 저장
 * 주어진 mc_id(설비 ID),  tool_no(공구 번호), data_type(대푯값 데이터 타입), stat_type(통계값 데이터 타입)에 대한 threshold 기준으로 이상 판별

In [427]:
def update_shot_result_db(engine, df_shot, report_result=False):
    if len(df_shot) < 1:
        return
    
    with engine.connect() as conn:
        query = ""
        alarm_state = 0
        prev_anomaly = 0

        #tbShotResult에 저장된 마지막 shot 정보를 가져오기
        df_last_shot = get_last_shot_info_at_shot_result_db(engine, machine_id=df_shot.iloc[0]['mc_id'], tool_no=df_shot.iloc[0]['tool_number'])

        if len(df_last_shot) > 0:
            prev_anomaly = df_last_shot.iloc[0]['anomaly_predict']

        normal_count = 0
        
        for index, row in df_shot.iterrows():
            alarm_state = 0

            if (prev_anomaly == 0) and (prev_anomaly != row['anomaly']) and (normal_count > 3):
                alarm_state = 1
            
            if row['anomaly'] == 0:
                if prev_anomaly == 1:
                    normal_count = 0
                else:
                    normal_count += 1

            prev_anomaly = row['anomaly']
            
            query = "SELECT id FROM tbShotResult where date = '{0}' and datetime = '{1}' and mc_id = {2} and tool_number = {3}\
                    and shot_no = {4} and shot_no_sub = {5}".format(row['date'], row['datetime'], row['mc_id'], row['tool_number'], row['shot_no'],
                                                              row['shot_no_sub'])
            result = conn.execute(query)
            
            #DB에 존재하지 않는 경우에만 DB에 추가 등록
            if result.rowcount < 1:
                query = "INSERT INTO tbShotResult (date, datetime, mc_id, tool_number, shot_no, shot_no_sub, anomaly_predict, alarm_state) VALUES\
                        ('{0}', '{1}', {2}, {3}, {4}, {5}, {6}, {7})".format(row['date'], row['datetime'], row['mc_id'], row['tool_number'], row['shot_no'],
                                                                             row['shot_no_sub'], row['anomaly'], alarm_state)
                if report_result is True:
                    print(query)

                conn.execute(query)

                query = "SELECT id FROM tbShotResult where date = '{0}' and datetime = '{1}' and mc_id = {2} and tool_number = {3}\
                        and shot_no = {4} and shot_no_sub = {5}".format(row['date'], row['datetime'], row['mc_id'], row['tool_number'], row['shot_no'],
                                                                  row['shot_no_sub'])
                if report_result is True:
                    print(query)

                result = conn.execute(query)
                
            result_id = result.first()[0]
            
            if report_result is True:
                print(result_id)
            
            query = "UPDATE tbShotStatistics SET result_id = {0} where date = '{1}' and datetime = '{2}' and mc_id = {3} and tool_number = {4}\
                    and shot_no = {5} and shot_no_sub = {6}".format(result_id, row['date'], row['datetime'], row['mc_id'], row['tool_number'], row['shot_no'],
                                                               row['shot_no_sub'])
            if report_result is True:
                print(query)
                
            conn.execute(query)
        conn.close()

### 주어진 mc_id(설비 ID),  tool_no(공구 번호), data_type(대푯값 데이터 타입)에 대한 threshold 기준으로 이상 판별
  * data_type : 대표값 데이터 타입 (current_spindle : 0, current_x = 1, current_z = 2)

In [428]:
def run_anomaly_detaction_for_all_shot(machine_id=22, tool_no=5, data_type_name ='current_spindle', stat_type_name='data_mean', report_result=False):
    #data type 판별
    data_type = 0
    
    if data_type_name == 'current_spindle':
        data_type = 0
    elif data_type_name == 'current_x':
        data_type = 1
    elif data_type_name == 'current_z':
        data_type = 2
    else:
        return
        
    engine = connect_db()
    
    #LJY_20221022 : tbShotResult에 저장된 마지막 shot 정보를 가져오기
    df_last_shot = get_last_shot_info_at_shot_result_db(engine, machine_id=machine_id, tool_no=tool_no)
    
    res = False

    #shot 대푯값 추출 결과 테이블(tbShotResult)가 비어있는 경우 TPOP_MACHINE_PARAMETER 테이블의 전체 데이터 중
    #  machine_id가 일치하는 shot list 추출
    if len(df_last_shot) < 1:
        if report_result is True:
            print("Need to run anomaly detaction for all shots with machine id {0}, tool_no {1}".format(machine_id, tool_no))
            
        df_shot_statistics = get_all_shot_specific_statistic_data_from_db_by_data_type_name(engine, machine_id, tool_no, data_type_name, stat_type_name)
    else:
        if report_result is True:
            print("Last shot from shot results table(tbShotResult) is -----\n{0}", df_last_shot.to_string())
            
        df_shot_statistics = get_all_shot_specific_stat_data_after_given_prev_data(engine, df_last_shot.iloc[0]['datetime'],
                                                df_last_shot.iloc[0]['shot_no'], machine_id, tool_no, data_type, stat_type_name)
        
        if report_result is True:
            print(df_shot_statistics)
    
    threshold = get_threshold_value_by_data_type_name(engine, machine_id, tool_no, data_type_name, stat_type_name, report_result)
    
    df_shot_statistics['anomaly'] = 0
    
    for index, row in df_shot_statistics.iterrows():
        if row[stat_type_name] >= threshold:
            df_shot_statistics.iloc[index, df_shot_statistics.columns.get_loc('anomaly')] = 1
            
            if report_result is True:
                print("row[{0}][{1}](value:{2}) >= threshold(value:{3})".format(index, stat_type_name, row[stat_type_name], threshold))
        else:
            if report_result is True:
                print("row[{0}][{1}](value:{2}) < threshold(value:{3})".format(index, stat_type_name, row[stat_type_name], threshold))
    
    #print(df_shot_statistics.query("anomaly == 1"))
    #print(df_shot_statistics.info())
    
    #DB에 shot 이상판별 결과 저장
    update_shot_result_db(engine, df_shot_statistics, report_result=report_result)
    
    del df_shot_statistics
    del df_last_shot
    gc.collect()

    engine.dispose()

# shot 별 대푯값 테이블 업데이트

In [429]:
extract_and_update_representatives_for_all_shot_with_db(['current_spindle'], report_result=True)

Last shot from shot results table(tbShotResult) is -----
{0}              datetime  shot_no  shot_no_sub  mc_id  tool_number
0 2022-10-31 11:10:14     7878            1     22            5
             datetime  shot_no  shot_no_sub  mc_id  tool_number
0 2022-10-31 11:10:14     7878            1     22            5
loop 0, shot_no 7879 extract


# shot 별 대푯값 기준으로 이상 판별 후 결과 기록

In [430]:
run_anomaly_detaction_for_all_shot()

   id        date            datetime  mc_id  tool_number  data_type  \
0   1  2022-10-31 2022-10-31 17:01:38     22            5          0   

   stat_type  threshold  
0  data_mean    2628.96  
